# modules

In [1]:
import numpy as np
import pandas as pd

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

import matplotlib.pyplot as plt

# data

In [2]:
!pwd

/c/Users/imed/Desktop/SID/project/CNN-HPO-Carbon/src/surrogate_modeling/rbf/playground


In [8]:
df = pd.read_csv('../../../../dataset/surrogate.csv')

# process

In [9]:
cat_cols = df.select_dtypes(include=['object']).columns

In [10]:
df = pd.get_dummies(df, columns=cat_cols)

# split

In [23]:
X = df.drop(columns=['train_accuracy', 'test_accuracy'])
y = df['test_accuracy']
y_ = df['train_accuracy']

In [24]:
X.shape, y.shape

((63, 123), (63,))

In [25]:
y.mean(), y.std(), y_.mean(), y_.std()

(np.float64(71.28746031746032),
 np.float64(5.23196193641019),
 np.float64(72.69552380952379),
 np.float64(5.741509621809161))

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [15]:
X_train = X
X_val = X
y_train = y
y_val = y

In [27]:
selector = VarianceThreshold(threshold=0.01)
X_train = selector.fit_transform(X_train)
X_val = selector.transform(X_val)

In [28]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [29]:
pca = PCA(n_components=0.9)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)

In [30]:
X_train.shape, X_val.shape

((56, 32), (7, 32))

# model

In [53]:
kernel = C(1, (1e-3, 1e3)) * RBF(length_scale=1, length_scale_bounds=(1e-3, 1e3))

gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-3, normalize_y=True, n_restarts_optimizer=200)

gpr.fit(X_train, y_train)
print("Fitted kernel:", gpr.kernel_)

Fitted kernel: 1**2 * RBF(length_scale=4.08)


In [66]:
y_pred_val, y_std_val = gpr.predict(X_val, return_std=True)
mse = mean_squared_error(y_val, y_pred_val)
print(f"\nValidation RMSE: {np.sqrt(mse):.4f}\n")


Validation RMSE: 4.3904



In [ ]:
%pip install optuna

In [67]:
import optuna


def objective(trial):

    constant_value = trial.suggest_float("constant_value", 0.1, 2, log=True)
    length_scale   = trial.suggest_float("length_scale",   0.1, 2, log=True)
    alpha          = trial.suggest_float("alpha",          1e-4, 1e-2, log=True)
    constant_bound_min = trial.suggest_float("constant_bound_min", 1e-5, 1e-2, log=True)
    constant_bound_max = trial.suggest_float("constant_bound_max", 1e2, 1e5, log=True)
    len_bound_min = trial.suggest_float("len_bound_min", 1e-5, 1e-2, log=True)
    len_bound_max = trial.suggest_float("len_bound_max", 1e2, 1e5, log=True)
    n_restarts = trial.suggest_int("n_restarts", 1, 20, log=True)

    kernel = C(constant_value, (constant_bound_min, constant_bound_max)) * RBF(length_scale, (len_bound_min, len_bound_max))
    gpr = GaussianProcessRegressor(
        kernel=kernel,
        alpha=alpha,
        normalize_y=True,
        n_restarts_optimizer=n_restarts,
        random_state=42
    )
    gpr.fit(X_train, y_train)

    y_pred, _ = gpr.predict(X_val, return_std=True)
    return np.sqrt(mean_squared_error(y_val, y_pred))


study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=200, timeout=600)  # e.g. 50 trials or 10 minutes


print(f"Finished {len(study.trials)} trials")
print("Best trial:")
print(f"  MSE:   {study.best_trial.value:.4f}")
print("  Params:")
for k, v in study.best_trial.params.items():
    print(f"    {k}: {v}")


best_params = study.best_trial.params
best_kernel = C(best_params["constant_value"], (best_params["constant_bound_min"], best_params["constant_bound_max"])) * RBF(best_params["length_scale"], (best_params["len_bound_min"], best_params["len_bound_max"]))
best_gpr = GaussianProcessRegressor(
    kernel=best_kernel,
    alpha=best_params["alpha"],
    normalize_y=True,
    n_restarts_optimizer=best_params["n_restarts"],
    random_state=42
)
best_gpr.fit(X_train, y_train)
print("Fitted kernel:", best_gpr.kernel_)
# X_combined = np.vstack([X_train, X_val])
# y_combined = np.concatenate([y_train, y_val])
# best_gpr.fit(X_combined, y_combined)
# print("Refit complete. Final kernel:", best_gpr.kernel_)


[I 2025-05-03 12:24:13,591] A new study created in memory with name: no-name-dcc7ad00-6407-4618-8880-add069c62eba
c:\Users\imed\Desktop\SID\ML\.venv\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 2.9375384576328295e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[I 2025-05-03 12:24:14,251] Trial 0 finished with value: 5.002494330000157 and parameters: {'constant_value': 0.30710573677773717, 'length_scale': 1.7254716573280358, 'alpha': 0.0029106359131330704, 'constant_bound_min': 0.0006251373574521745, 'constant_bound_max': 293.8027938703535, 'len_bound_min': 2.9375384576328295e-05, 'len_bound_max': 149.36568554617628, 'n_restarts': 12}. Best is trial 0 with value: 5.002494330000157.
[I 2025-05-03 12:24:14,281] Trial 1 finished with value: 5.002494330000157 and parameters: {'constant_value': 0.60543658554692

Finished 200 trials
Best trial:
  MSE:   4.3900
  Params:
    constant_value: 0.9581790360612081
    length_scale: 0.156802600198971
    alpha: 0.00010323493569889644
    constant_bound_min: 1.2036239471658982e-05
    constant_bound_max: 2774.319212662227
    len_bound_min: 1.0217352329359807e-05
    len_bound_max: 84756.61491213234
    n_restarts: 6
Fitted kernel: 1**2 * RBF(length_scale=4.07)


In [68]:
y_pred_val, y_std_val = best_gpr.predict(X_val, return_std=True)
mse = mean_squared_error(y_val, y_pred_val)
print(f"\nValidation RMSE: {np.sqrt(mse):.4f}\n")


Validation RMSE: 4.3900



In [69]:
y_pred_val

array([71.93693388, 70.4961909 , 72.13216883, 70.83149895, 70.92049944,
       71.04589048, 69.93767765])

In [43]:
X.columns.to_list()

['initial_conv_hp_channels',
 'initial_conv_hp_kernel_size',
 'initial_conv_hp_stride',
 'block_0_expanded_channels',
 'block_0_use_se',
 'block_0_se_squeeze_factor',
 'block_0_channels',
 'block_0_kernel_size',
 'block_0_stride',
 'block_1_expanded_channels',
 'block_1_use_se',
 'block_1_se_squeeze_factor',
 'block_1_channels',
 'block_1_kernel_size',
 'block_1_stride',
 'block_2_expanded_channels',
 'block_2_use_se',
 'block_2_se_squeeze_factor',
 'block_2_channels',
 'block_2_kernel_size',
 'block_2_stride',
 'block_3_expanded_channels',
 'block_3_use_se',
 'block_3_se_squeeze_factor',
 'block_3_channels',
 'block_3_kernel_size',
 'block_3_stride',
 'block_4_expanded_channels',
 'block_4_use_se',
 'block_4_se_squeeze_factor',
 'block_4_channels',
 'block_4_kernel_size',
 'block_4_stride',
 'block_5_expanded_channels',
 'block_5_use_se',
 'block_5_se_squeeze_factor',
 'block_5_channels',
 'block_5_kernel_size',
 'block_5_stride',
 'block_6_expanded_channels',
 'block_6_use_se',
 'blo

In [70]:
confidence_z = {
    0.40: 0.524,   # 40%
    0.50: 0.674,   # 50%
    0.60: 0.841,   # 60%
    0.70: 1.036,   # 70%
    0.80: 1.282,   # 80%
    0.85: 1.440,   # 85%
    0.90: 1.645,   # 90%
    0.95: 1.960,   # 95%
    0.98: 2.326,   # 98%
    0.99: 2.576,   # 99%
    0.999: 3.291,  # 99.9%
    0.9999: 3.891  # 99.99%
}

In [71]:
confidence = 0.9
z = confidence_z[confidence]
print(f"Confidence level: {confidence*100:.1f}%")

for x_val, y_true in zip(X_val, y_val):
    x_val = x_val.reshape(1, -1)
    y_pred, y_std = gpr.predict(x_val, return_std=True)
    mean = y_pred[0]
    std = y_std[0]
    ci = z * std
    print(f"Val Sample:  True = {y_true:.3f}, Predicted = {mean:.3f}, CI ≈ [{mean - ci:.3f}, {mean + ci:.3f}]")

Confidence level: 90.0%
Val Sample:  True = 67.440, Predicted = 71.937, CI ≈ [63.458, 80.415]
Val Sample:  True = 63.400, Predicted = 70.497, CI ≈ [61.966, 79.027]
Val Sample:  True = 69.890, Predicted = 72.132, CI ≈ [63.596, 80.667]
Val Sample:  True = 71.760, Predicted = 70.832, CI ≈ [62.299, 79.365]
Val Sample:  True = 65.660, Predicted = 70.921, CI ≈ [62.403, 79.439]
Val Sample:  True = 73.410, Predicted = 71.046, CI ≈ [62.623, 79.469]
Val Sample:  True = 64.920, Predicted = 69.939, CI ≈ [61.719, 78.158]


In [72]:
from scipy.stats import norm

confidence = 0.9
z = confidence_z[confidence]
print(f"Confidence level: {confidence*100:.1f}%")

kappa = z
xi = 0.01
y_best = max(y_train)

for x_val, y_true in zip(X_val, y_val):
    x_val = x_val.reshape(1, -1)
    y_pred, y_std = gpr.predict(x_val, return_std=True)
    mean = y_pred[0]
    std = y_std[0]
    ci = z * std
    ucb = mean + kappa * std
    if std == 0:
        ei = 0.0
    else:
        z_ei = (mean - y_best - xi) / std
        ei = (mean - y_best - xi) * norm.cdf(z_ei) + std * norm.pdf(z_ei)

    print(
        f"True: {y_true:.3f} | Pred: {mean:.3f} | CI ±{ci:.3f} → [{mean - ci:.3f}, {mean + ci:.3f}]"
        f" | UCB: {ucb:.3f} | EI: {ei:.6f}"
    )

Confidence level: 90.0%
True: 67.440 | Pred: 71.937 | CI ±8.479 → [63.458, 80.415] | UCB: 80.415 | EI: 0.116721
True: 63.400 | Pred: 70.497 | CI ±8.531 → [61.966, 79.027] | UCB: 79.027 | EI: 0.060557
True: 69.890 | Pred: 72.132 | CI ±8.535 → [63.596, 80.667] | UCB: 80.667 | EI: 0.131624
True: 71.760 | Pred: 70.832 | CI ±8.533 → [62.299, 79.365] | UCB: 79.365 | EI: 0.071532
True: 65.660 | Pred: 70.921 | CI ±8.518 → [62.403, 79.439] | UCB: 79.439 | EI: 0.073963
True: 73.410 | Pred: 71.046 | CI ±8.423 → [62.623, 79.469] | UCB: 79.469 | EI: 0.073919
True: 64.920 | Pred: 69.939 | CI ±8.219 → [61.719, 78.158] | UCB: 78.158 | EI: 0.035914


In [82]:
from sklearn.linear_model import Ridge, Lasso, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

# Optional: add more models here
regressors = {
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "BayesianRidge": BayesianRidge(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(n_neighbors=6),
    "DecisionTree": DecisionTreeRegressor(),
}

print("Benchmarking regressors...\n")

results = []

for name, model in regressors.items():
    # Wrap with a pipeline that includes feature scaling
    pipe = make_pipeline(StandardScaler(), model)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    results.append((name, rmse))
    print(f"{name:<18} RMSE: {rmse:.4f}")


Benchmarking regressors...

Ridge              RMSE: 4.5738
Lasso              RMSE: 4.5055
BayesianRidge      RMSE: 4.8840
RandomForest       RMSE: 4.4320
GradientBoosting   RMSE: 4.5788
SVR                RMSE: 5.8962
KNN                RMSE: 4.0391
DecisionTree       RMSE: 5.3660
